In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
# 1638 words
with open("../Data/wordpools/wasnorm_wordpool.txt") as f:
    items = f.readlines()
items = [i.replace("\n","") for i in items]
items = np.array(items)

In [3]:
simu_sess_num = 10000
n = 6
wordpool = np.arange(1,1639)

In [4]:
df_study = pd.DataFrame()
df_test = pd.DataFrame()
rng = np.random.default_rng(seed=42)

for sess in range(simu_sess_num):
    tmp_df_study = pd.DataFrame()
    tmp_df_test = pd.DataFrame()
    
    # choose present words
    pres_words = rng.choice(wordpool, 2*n, replace = False)
    pres_words = np.reshape(pres_words, (n,2))
    tmp_df_study['study_itemno1'] = pres_words[:,0]
    tmp_df_study['study_itemno2'] = pres_words[:,1]
    tmp_df_study['study_item1'] = items[pres_words[:,0]-1]
    tmp_df_study['study_item2'] = items[pres_words[:,1]-1]
    tmp_df_study['list'] = 1
    tmp_df_study['session'] = sess
    df_study = pd.concat([df_study,tmp_df_study])
    
    # choose test words
    order = np.arange(5,-1,-1)
    direction = rng.permutation([0] * 3 + [1] * 3)
    test_probe = pres_words[order,direction]
    correct_ans = pres_words[order,1-direction]
    tmp_df_test['test_itemno'] = test_probe
    tmp_df_test['test_item'] = items[test_probe-1]
    tmp_df_test['correct_ans'] = correct_ans
    tmp_df_test['lag'] = np.arange(0,12,2)
    tmp_df_test['list'] = 1
    tmp_df_test['session'] = sess
    tmp_df_test['order'] = direction + 1  # 1 = forward, 2 = backward
    df_test = pd.concat([df_test,tmp_df_test])

df_study.reset_index(inplace = True, drop=True)
df_test.reset_index(inplace = True, drop=True)  

In [5]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,list,session
0,716,141,HOOP,BLUEJAY,1,0
1,1261,862,SEAGULL,MAN,1,0
2,1067,1402,PIKE,SUIT,1,0
3,146,707,BOLT,HIGHWAY,1,0
4,155,1140,BOULDER,PUDDLE,1,0
...,...,...,...,...,...,...
59995,1402,1501,SUIT,TREAT,1,9999
59996,101,473,BATTERY,DISC,1,9999
59997,345,980,COLLAR,ORANGE,1,9999
59998,407,1360,CRIB,SPY,1,9999


In [6]:
df_test

,test_itemno,test_item,correct_ans,lag,list,session,order
0,330,CLOWN,1599,0,1,0,2
1,155,BOULDER,1140,2,1,0,1
2,707,HIGHWAY,146,4,1,0,2
3,1067,PIKE,1402,6,1,0,1
4,862,MAN,1261,8,1,0,2
...,...,...,...,...,...,...,...
59995,1360,SPY,407,2,1,9999,2
59996,980,ORANGE,345,4,1,9999,2
59997,101,BATTERY,473,6,1,9999,1
59998,1402,SUIT,1501,8,1,9999,1


In [7]:
# save
with open('../Data/simu6a1_design.pkl', 'wb') as outp:
    pickle.dump(df_study, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(df_test, outp, pickle.HIGHEST_PROTOCOL)